In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import joblib


# 1. Завантаження даних

data = pd.read_csv("weatherAUS.csv")

data = data.dropna(subset=["RainTomorrow"])


# 2. Визначення колонок

target_col = "RainTomorrow"
categorical_cols = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']
numeric_cols = [
    'MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine',
    'WindGustSpeed','WindSpeed9am','WindSpeed3pm',
    'Humidity9am','Humidity3pm',
    'Pressure9am','Pressure3pm',
    'Cloud9am','Cloud3pm',
    'Temp9am','Temp3pm'
]
input_cols = categorical_cols + numeric_cols

X = data[input_cols]
y = (data[target_col] == "Yes").astype(int)


# 3. Препроцесинг

imputer = SimpleImputer()
scaler = MinMaxScaler()
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Імпутація
X[numeric_cols] = imputer.fit_transform(X[numeric_cols])

# Масштабування числових
X_scaled = scaler.fit_transform(X[numeric_cols])
X_scaled = pd.DataFrame(X_scaled, columns=numeric_cols)

# Кодування категоріальних
X_encoded = encoder.fit_transform(X[categorical_cols])
encoded_cols = encoder.get_feature_names_out(categorical_cols)
X_encoded = pd.DataFrame(X_encoded, columns=encoded_cols)

# Об’єднання
X_ready = pd.concat([X_scaled, X_encoded], axis=1)


# 4. Тренування моделей

X_train, X_test, y_train, y_test = train_test_split(X_ready, y, test_size=0.2, random_state=42)

# --- Logistic Regression
log_model = LogisticRegression(max_iter=500, solver='liblinear')
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)
y_prob_log = log_model.predict_proba(X_test)[:, 1]

# --- Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]


# 5. Порівняння результатів

print("📈 Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, y_prob_log))
print(classification_report(y_test, y_pred_log))

print("🌲 Random Forest")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_prob_rf))
print(classification_report(y_test, y_pred_rf))


# 6. Збереження кращої моделі 

# Якщо Random Forest має вищий ROC AUC:
if roc_auc_score(y_test, y_prob_rf) > roc_auc_score(y_test, y_prob_log):
    best_model = rf_model
    model_name = "aussie_rain_rf.joblib"
    print("\n✅ Random Forest обрано як кращу модель.")
else:
    best_model = log_model
    model_name = "aussie_rain_log.joblib"
    print("\n✅ Logistic Regression залишилась кращою.")

joblib.dump({
    'model': best_model,
    'imputer': imputer,
    'scaler': scaler,
    'encoder': encoder,
    'input_cols': input_cols,
    'numeric_cols': numeric_cols,
    'categorical_cols': categorical_cols
}, model_name)
print(f"💾 Модель збережено як {model_name}")

/var/folders/m_/08q67_2569g7rxl_d8m9kk2r0000gn/T/ipykernel_68166/3237520122.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols] = imputer.fit_transform(X[numeric_cols])


📈 Logistic Regression
Accuracy: 0.846337775589859
ROC AUC: 0.8681568772465998
              precision    recall  f1-score   support

           0       0.87      0.94      0.90     22098
           1       0.72      0.51      0.60      6341

    accuracy                           0.85     28439
   macro avg       0.79      0.73      0.75     28439
weighted avg       0.84      0.85      0.84     28439

🌲 Random Forest
Accuracy: 0.8530187418685607
ROC AUC: 0.883297686900558
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     22098
           1       0.76      0.50      0.60      6341

    accuracy                           0.85     28439
   macro avg       0.81      0.73      0.76     28439
weighted avg       0.84      0.85      0.84     28439


✅ Random Forest обрано як кращу модель.
💾 Модель збережено як aussie_rain_rf.joblib
